** Analysis planning **

The analysis performed by delentture will be the same as performed in the first part of the project.

That being said, the columns starting with "past_" will be the key information to be passed, as well as the age

In [15]:
import pandas as pd
import pydp as dp
from pydp.algorithms.numerical_mechanisms import NumericalMechanism
from pydp.algorithms.laplacian import BoundedSum, BoundedMean, Count, Max


In [16]:
df = pd.read_csv("infringement_dataset_v2.csv")

In [17]:


# row = person, column = values
# locate row: df.loc[1]
# selecionar as linhas que infrigiram e que não infrigiram
# cria uma Dataframe para cada caso
sel1 = df[df["infringed"] == 1]
sel2 = df[df["infringed"] == 0]

# definir variáveis para os gráficos

past_annuity = [sel2["past_avg_amount_annuity"].mean(), sel1["past_avg_amount_annuity"].mean()]
past_application = [sel2["past_avg_amt_application"].mean(), sel1["past_avg_amt_application"].mean()]
past_credit = [sel2["past_avg_amt_credit"].mean(), sel1["past_avg_amt_credit"].mean()]
past_approved = [sel2["past_loans_approved"].mean(), sel1["past_loans_approved"].mean()]
past_cancelled = [sel2["past_loans_canceled"].mean(), sel1["past_loans_canceled"].mean()]
past_refused = [sel2["past_loans_refused"].mean(), sel1["past_loans_refused"].mean()]
past_total = [sel2["past_loans_total"].mean(), sel1["past_loans_total"].mean()]
past_unused = [sel2["past_loans_unused"].mean(), sel1["past_loans_unused"].mean()]

age1 = df[df["age"] < 30].size
age2 = df[df["age"] < 40].size - age1
age3 = df[df["age"] < 50].size - age2
age4 = df[df["age"] < 60].size - age3
age5 = df[df["age"] >= 60].size


In [23]:
dp_sum = BoundedSum(epsilon= 1.5, lower_bound =  5, upper_bound = 250, dtype ='float')
dp_sum_og = dp_sum.quick_result(df['age'].to_list())
print(dp_sum_og)

13357248.793884434
